# Blowing the doors off your bottlenecks with Python on AMD APUs and dGPUs
## Stan Seibert<br/>December 8, 2015 (updated March 2016)

(This notebook contains code from a webinar showing how to use Python and Numba to write code that takes advantage of the GPU-capabilities on AMD APUs, it has subsequently been modified to also demonstrate AMD dGPUs.)

This example requires:
 * One of: 
     * An AMD APU: Kaveri and Carrizo processors have been tested
     * A >= Haswell Intel CPU with an AMD dGPU, the R9 Nano dGPU has been tested.
 * Ubuntu Linux 14.04 64-bit
 * Drivers/Runtime: 
     * For AMD APUs: [AMD HSA 1.0 final drivers](https://github.com/HSAFoundation/HSA-Docs-AMD/wiki/HSA-Platforms-&-Installation)
     * For AMD dGPUs (All available as part of  [Radeon Open Compute Solutions](https://github.com/RadeonOpenCompute)):
         * [ROCR-Runtime](https://github.com/RadeonOpenCompute/ROCR-Runtime)
         * [ROCK-Kernel-Driver](https://github.com/RadeonOpenCompute/ROCK-Kernel-Driver)
         * [ROCT-Thunk-Interface](https://github.com/RadeonOpenCompute/ROCT-Thunk-Interface)
         
         

The easiest way to test out the code in this notebook is to download [Miniconda](http://conda.pydata.org/miniconda.html) and run the following commands to create and activate a new environment:
```
conda create -n hsa_webinar python=3.4 numba libhlc pandas bokeh matplotlib basemap jupyter
source activate hsa_webinar
export LD_LIBRARY_PATH=/opt/hsa/lib

jupyter notebook
```
TODO : check the above for dGPU with respect to libHLC status.

Set up... select a floating point type and, if there is a dGPU, whether to use preallocated memory in the performance tests.

In [ ]:
import numpy as np

# The data type to use in the computation, `np.float64` and `np.float32` are valid.
global_dtype = np.float64

# Whether to use automatic device allocation or pre-allocated memory (dGPU option only).
use_auto_device = False


In [ ]:
from numba.hsa.hsadrv.driver import dgpu_present
from numba.hsa.hsadrv.devices import get_context
import numba.hsa.api as hsaapi
gt_as_str = global_dtype.__name__
dgpu_prealloc = False
if use_auto_device == False:
    if dgpu_present() == True:
        dgpu_prealloc = True
    else:
        raise RuntimeError("dGPU preallocation requested but no dGPU is found.")

In [ ]:
%matplotlib inline
import pandas as pd
import numba.hsa
from bokeh.plotting import output_notebook, figure, show
from bokeh.models import NumeralTickFormatter
output_notebook()

## Sample Data

For these examples, we use geographic points information recorded by a NASA satellite observing lightning strikes from orbit.  These points are excerpted from the [LIS/OTD Climatology dataset](http://thunder.msfc.nasa.gov/data/data_lis-otd-climatology.html).

The data have been saved in compressed CSV form in the Git repository.

In [ ]:
lstrikes = pd.read_csv('points.csv.gz')
print('Number of points in file:', len(lstrikes))
lstrikes = pd.concat([lstrikes]*21) # Expand the test data by duplicating it 21 times (~2**22 points)
print('Number of points in memory:', len(lstrikes))

We can draw a map showing a subset of these values:

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 12, 6

m = Basemap(projection='merc',llcrnrlat=26,urcrnrlat=37,\
            llcrnrlon=-107,urcrnrlon=-90,lat_ts=1, resolution='l')
# draw coastlines.
m.drawcoastlines(color='#253746')
m.drawcountries(color='#253746')
m.drawstates(color='#253746')
m.drawmapboundary(fill_color='#089bcc')
m.fillcontinents(color='#A4BCC2',lake_color='#089bcc')

sample = lstrikes.sample(n=1000)
m.scatter(y=sample.latitude.values, x=sample.longitude.values, latlon=True, marker='.', color='yellow')
m.plot(y=30.25, x=-97.75, latlon=True, marker="*", color="#253746")
m.tissot(-97.75, 30.25, 3.59, npts=30, alpha=0.20, facecolor='#089bcc', edgecolor='#253746', linewidth='3')
plt.show()

## Example 1:  Creating a Ufunc

[Universal functions](http://docs.scipy.org/doc/numpy-1.10.0/reference/ufuncs.html) are an incredibly useful concept from NumPy for describing operations on arrays of data.  A simple example using two ufuncs is:

In [ ]:
a = np.array([1, 4, 9])
result = np.sqrt(a) + 1
print(result)

Let's first create a function that computes the distance between two points on the surface of the earth:

In [ ]:
import math

R_EARTH = 6371.0 # km

sig='%s(%s)' % ((gt_as_str,)*2)

@numba.hsa.jit(sig, device=True)
def deg2rad(deg):
    return math.pi * deg / 180.0

sig='%s(%s, %s, %s, %s)' % ((gt_as_str,)*5)
@numba.vectorize(sig, target='hsa')
def gpu_great_circle_distance(lat1, lng1, lat2, lng2):
    '''Return the great-circle distance in km between (lat1, lng1) and (lat2, lng2)
    on the surface of the Earth.'''
    lat1, lng1 = deg2rad(lat1), deg2rad(lng1)
    lat2, lng2 = deg2rad(lat2), deg2rad(lng2)

    sin_lat1, cos_lat1 = math.sin(lat1), math.cos(lat1)
    sin_lat2, cos_lat2 = math.sin(lat2), math.cos(lat2)

    delta = lng1 - lng2
    sin_delta, cos_delta = math.sin(delta), math.cos(delta)

    numerator = math.sqrt((cos_lat1 * sin_delta)**2 + 
                          (cos_lat1 * sin_lat2 - sin_lat1 * cos_lat2 * cos_delta)**2)
    denominator = sin_lat1 * sin_lat2 + cos_lat1 * cos_lat2 * cos_delta
    return R_EARTH * math.atan2(numerator, denominator)

Next we can verify that it works, and see how fast it is for a particular data set size:

In [ ]:
print('Distance between Austin and Chicago:', gpu_great_circle_distance(30.25, -97.75, 41.8369, -87.6847)[0], 'km')

In [ ]:
c7_lat = lstrikes.latitude.values.astype(global_dtype)
c7_long = lstrikes.longitude.values.astype(global_dtype)
%timeit distances = gpu_great_circle_distance(30.25, -97.75, c7_lat, c7_long)

To benchmark the HSA version of the algorithm, we need a NumPy version to compare with:

In [ ]:
def numpy_great_circle_distance(lat1, lng1, lat2, lng2):
    '''Return the great-circle distance in km between (lat1, lng1) and (lat2, lng2)
    on the surface of the Earth.'''
    lat1, lng1 = np.deg2rad(lat1), np.deg2rad(lng1)
    lat2, lng2 = np.deg2rad(lat2), np.deg2rad(lng2)

    # See last formula in http://en.wikipedia.org/wiki/Great-circle_distance#Computational_formulas
    sin_lat1 = np.sin(lat1)
    cos_lat1 = np.cos(lat1)
    sin_lat2 = np.sin(lat2)
    cos_lat2 = np.cos(lat2)

    delta = lng1 - lng2
    cos_delta = np.cos(delta)
    sin_delta = np.sin(delta)

    numerator = np.sqrt((cos_lat1 * sin_delta)**2 + 
                        (cos_lat1 * sin_lat2 - sin_lat1 * cos_lat2 * cos_delta)**2)
    denominator = sin_lat1 * sin_lat2 + cos_lat1 * cos_lat2 * cos_delta
    return R_EARTH * np.arctan2(numerator, denominator)

Finally, we can benchmark for a range of data sizes:

In [ ]:
target_lat, target_lng = 30.25, -97.75

cpu = []
hsa = []
ex1_n = []

if dgpu_prealloc:
    top_end = 23
else:
    top_end = 21

for lg2 in range(8, top_end):
    npoints = 2**lg2
    print(npoints)
    ex1_n.append(npoints)
    sample = lstrikes.iloc[:npoints]
    lat = sample.latitude.values.astype(global_dtype)
    lng = sample.longitude.values.astype(global_dtype)
    result = %timeit -o numpy_great_circle_distance(target_lat, target_lng, lat, lng)
    cpu.append(result.best)
    if dgpu_prealloc:
        ctx = get_context()
        d_lat = hsaapi.to_device(lat, ctx)
        d_lng = hsaapi.to_device(lng, ctx)
        res = np.zeros(npoints, dtype=global_dtype)
        d_res = hsaapi.to_device(res, ctx)
        result = %timeit -o gpu_great_circle_distance(target_lat, target_lng, d_lat, d_lng, out = d_res)
    else:
        result = %timeit -o gpu_great_circle_distance(target_lat, target_lng, lat, lng)
    hsa.append(result.best)


# Compute speedup ratio
ex1_speedup = np.array(cpu) / np.array(hsa)

In [ ]:
p = figure(plot_width=800, plot_height=500, title='Compute Distance from Target')
p.xaxis.axis_label = 'Number of points'
p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
p.yaxis.axis_label = 'GPU speedup ratio'
p.line(ex1_n, ex1_speedup, color='navy')
p.line(ex1_n, 1.0, color='red', line_dash=[10,10])
show(p)

## Example 2: Creating an HSA kernel

In this next example, we will use HSA to compute the distance matrix between a set of points.

This device function computes the distance between two points.  (Very similar to what we used in our ufunc above.)

In [ ]:
sig='%s(%s, %s, %s, %s)' % ((gt_as_str,)*5)
@numba.hsa.jit(sig, device=True)
def device_great_circle_distance(lat1, lng1, lat2, lng2):
    lat1, lng1 = deg2rad(lat1), deg2rad(lng1)
    lat2, lng2 = deg2rad(lat2), deg2rad(lng2)

    sin_lat1 = math.sin(lat1)
    cos_lat1 = math.cos(lat1)
    sin_lat2 = math.sin(lat2)
    cos_lat2 = math.cos(lat2)

    delta = lng1 - lng2
    cos_delta = math.cos(delta)
    sin_delta = math.sin(delta)

    numerator = math.sqrt((cos_lat1 * sin_delta)**2 + 
                          (cos_lat1 * sin_lat2 - sin_lat1 * cos_lat2 * cos_delta)**2)
    denominator = sin_lat1 * sin_lat2 + cos_lat1 * cos_lat2 * cos_delta
    return R_EARTH * math.atan2(numerator, denominator)

This is the kernel that will map HSA work-items to point pairs in the distance calculation:

In [ ]:
sig='(%s[:], %s[:], %s[:,:])' % ((gt_as_str,)*3)
@numba.hsa.jit(sig)
def distance_matrix(lat, lng, distance):
    this_idx = numba.hsa.get_global_id(0)
    npoints = lat.size
    
    if this_idx >= npoints:
        return
    
    this_lat = lat[this_idx]
    this_lng = lng[this_idx]
    
    for other_idx in range(npoints):
        other_lat = lat[other_idx]
        other_lng = lng[other_idx]
        
        distance[this_idx, other_idx] = device_great_circle_distance(this_lat, this_lng,
                                                                     other_lat, other_lng)

And here is how to launch a computation of the distance matrix for 4 points:

In [ ]:
# Create output array
sample = lstrikes.iloc[:4]
distance = np.empty(shape=(len(sample), len(sample)), dtype=global_dtype)

# Compute dimensions of grid
items_per_group = 64 # Use multiple of 64
groups = int(math.ceil(len(sample) / items_per_group))

# Call GPU kernel
distance_matrix[groups, items_per_group](sample.latitude.values.astype(global_dtype), 
                                         sample.longitude.values.astype(global_dtype),
                                         distance)
print(distance)

For comparison, we can compute the same matrix with NumPy:

In [ ]:
numpy_great_circle_distance(sample.latitude.values[:, np.newaxis].astype(global_dtype), 
                            sample.longitude.values[:, np.newaxis].astype(global_dtype),
                            sample.latitude.values[np.newaxis, :].astype(global_dtype),
                            sample.longitude.values[np.newaxis, :].astype(global_dtype))

Again, we can benchmark the distance matrix calculation for a range of input sizes.

In [ ]:
import gc
cpu = []
hsa = []
ex2_n = []

# more float32s can exist on the dGPU card than float64s before the gc gets stuck
if isinstance(global_dtype, np.float32):
    lim = 14
else:
    lim = 13
   
npoints_list = (2 ** np.array(range(8, lim))).astype(int).tolist()

for npoints in npoints_list:
    print(npoints)
    ex2_n.append(npoints)
    sample = lstrikes.iloc[:npoints]
    lat = sample.latitude.values.astype(global_dtype)
    lng = sample.longitude.values.astype(global_dtype)
    
    distance = np.empty(shape=(len(sample), len(sample)), dtype=global_dtype)

    lat_ax0 = lat[:, np.newaxis].astype(global_dtype)
    lng_ax0 = lng[:, np.newaxis].astype(global_dtype)
    lat_ax1 = lat[np.newaxis, :].astype(global_dtype)
    lng_ax1 = lng[np.newaxis, :].astype(global_dtype)
    if dgpu_prealloc:
        result = %timeit -o -n 10 numpy_great_circle_distance(\
            lat_ax0, lng_ax0, lat_ax1, lng_ax1)
    else:
        result = %timeit -o -n 1 numpy_great_circle_distance(\
            lat_ax0, lng_ax0, lat_ax1, lng_ax1)
    cpu.append(result.best)

    if dgpu_prealloc:
        ctx = get_context()
        d_lat = hsaapi.to_device(lat, ctx)
        d_lng = hsaapi.to_device(lng, ctx)
        d_distance = hsaapi.to_device(distance, ctx)
    
    items_per_group = 64 # Use multiple of 64
    groups = int(math.ceil(len(sample) / items_per_group))
    if dgpu_prealloc:
        result = %timeit -o -n 10 distance_matrix[groups, items_per_group](d_lat, d_lng, d_distance)
    else:
        result = %timeit -o -n 1 distance_matrix[groups, items_per_group](lat, lng, distance)
    gc.collect()
    hsa.append(result.best)

# Compute speedup ratio
ex2_speedup = np.array(cpu) / np.array(hsa)

In [ ]:
print(ex2_speedup)
p = figure(plot_width=800, plot_height=500, title='Compute Distance Matrix')
p.xaxis.axis_label = 'Number of points'
p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
p.yaxis.axis_label = 'GPU speedup ratio'
p.line(ex2_n, ex2_speedup, color='navy')
p.line(ex2_n, 1.0, color='red', line_dash=[10,10])
show(p)

## Other examples

### Compiling a ufunc for both CPU and GPU

In [ ]:
def reldiff(a, b):
    return 2 * (a - b) / (a + b)

sig='%s(%s, %s)' % ((gt_as_str,)*3)
cpu_reldiff = numba.vectorize(sig)(reldiff)
gpu_reldiff = numba.vectorize(sig, target='hsa')(reldiff)

In [ ]:
sz = 10000000
x = np.random.uniform(size=sz).astype(global_dtype)
y = np.random.uniform(size=sz).astype(global_dtype)

if dgpu_prealloc:
    ctx = get_context()
    d_x = hsaapi.to_device(x, ctx)
    d_y = hsaapi.to_device(y, ctx)
    d_res = hsaapi.to_device(np.zeros_like(x), ctx)

cpu_result = cpu_reldiff(x, y)
gpu_result = gpu_reldiff(x, y)

print('CPU:', cpu_result)
print('GPU:', gpu_result)

For extremely simple functions, like `reldiff`, the GPU does not offer much benefit *unless*
the device memory is preallocated, as is demonstrated in the case of a dGPU:

In [ ]:
%timeit cpu_reldiff(x, y)
if dgpu_prealloc:
    %timeit gpu_reldiff(d_x, d_y, out = d_res)
%timeit gpu_reldiff(x, y)